<a href="https://colab.research.google.com/github/LazyTurtleknight/deepl/blob/accelerate_performance/Training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas
!pip install albumentations
!pip install torchmetrics
!pip install tensorboard

In [4]:
import os
import albumentations as alb
#import tensorflow as tf
import datetime
import torch
import numpy as np
import opendatasets as od
import tensorboard

#from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from albumentations.pytorch.transforms import ToTensorV2
from torchmetrics.classification import Dice

In [ ]:
# Download dataset from kaggle
od.download(
    "https://www.kaggle.com/datasets/balraj98/deepglobe-land-cover-classification-dataset")

In [5]:
# model.py
import torch
import numpy as np

from torch import nn

# U-Net

# Two convolution block. Performs two consecutive convolutions
class TwoConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding='same'):
        super().__init__()

        self.module_list = nn.ModuleList([])

        #Using Henriks convultion layering or the one introduced in the Unet paper?
        self.module_list.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))
        self.module_list.append(nn.ReLU())

        self.module_list.append(nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding))
        self.module_list.append(nn.ReLU())

    def forward(self, x):
        y = x
        for module in self.module_list:
            y = module(y)
        return y

# UNet encoder block. Performs two convolutions and max pooling.
class ConvPool(TwoConv):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding='same'):
        super().__init__(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.max = nn.MaxPool2d(2, 2)

    def forward(self, x):
        c = super().forward(x)
        p = self.max(c)
        return c, p

# UNet decoder block. Performs upsampling, concatenation of the two inputs and two convolutions.
class UpConv(TwoConv):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding='same'):
        super().__init__(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        # We may use different upsampling method here.
        self.upsampling = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x, skip):
        u = self.upsampling(x)
        u = torch.cat([u, skip], 1)
        c = super().forward(u)
        return c, u


class UNet(nn.Module):
    def __init__(self, in_channels, min, max, num_classes):
        super().__init__()
        self.enc_layers = nn.ModuleList([])
        self.dec_layers = nn.ModuleList([])
        self.enc_final = None
        self.dec_final = None
        self.softmax = None

        # When go down the encoder/up the decoder the number of filter doubles/halves
        # respectively. For that we will generate the powers of two.
        # List of powers of 2 [min, 2*min, 4*min, ..., max]
        channels = []
        power = min
        for i in range(int(np.log2(max // min))):
            channels.append(power)
            power = power*2

        # Construct list of blocks for the encoder
        self.enc_layers.append(ConvPool(in_channels, min))
        for i in range(len(channels)-1):
            enc_layer = ConvPool(channels[i], channels[i+1])
            self.enc_layers.append(enc_layer)

        # Construct list of blocks for the encoder
        for i in range(len(channels)-1):
            dec_layer = UpConv(channels[i+1], channels[i])
            self.dec_layers.insert(0, dec_layer)
        self.dec_layers.insert(0, UpConv(max, channels[-1]))

        # Set up final convolutions for the encoder and decoder
        self.enc_final = TwoConv(channels[len(channels)-1], max, 3, 1, 'same')
        self.dec_final = nn.Conv2d(min, num_classes, 1, 1)
        self.softmax = nn.Softmax(0)

    def forward(self, x):
        # Collect the values for skip connections to the decoder
        skip_connections = []
        p = x
        # Encoder
        for layer in self.enc_layers:
            c, p = layer(p)
            skip_connections.append(c)

        # Bottleneck
        c =  self.enc_final(p)

        # Decoder
        for layer in self.dec_layers:
            skip = skip_connections.pop()
            c, u = layer(c, skip) # if we do not need c we can use _ instead
        c = self.dec_final(c)

        return self.softmax(c)

In [11]:
# dataset.py
#Custom dataset for SatelliteSet
import cv2
import pandas as pd
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset


# Custom dataset class to load deep globe dataset
class SatelliteSet(Dataset):
    def __init__(self,
                 # mpandas dataframe loaded with a meta data csv containing image file names
                 meta_data,
                 # Class dictionary
                 class_dict,
                 # directory where the data is stored
                 data_dir,
                 # albumentations transform
                 transform=None):
        self.meta_data = meta_data
        self.data_dir = data_dir
        self.transform = transform
        self.class_dict = class_dict

        """
         Note:
                - We load data completely into main memory and into gpu memory (half of the samples are on cuda, the other on main)
                - samples are cropped -> no random crop each epoch
                - idx_to_divice keeps track of where samples are (cuda or cpu)
        """
        self.idx_to_device = dict()
        # shape: (len(self) // 2) x 3 x 256 x 256
        self.images_cpu = None
        # shape: (len(self) // 2) x 7 x 256 x 256
        self.images_cuda = None
        # shape: (len(self) // 2) x 3 x 256 x 256
        self.masks_cpu = None
        # shape: (len(self) // 2) x 7 x 256 x 256
        self.masks_cuda = None
        self.prefetch()


    def print_memory(self):
        func = lambda t: print(t.element_size() * t.nelement())
        func(self.images_cpu)
        func(self.images_cuda)
        func(self.masks_cpu)
        func(self.masks_cuda)

    # number of samples in dataset
    def __len__(self):
        return len(self.meta_data)

    def prefetch(self):
        toggle = True
        for idx in range(len(self) // 2):
            # Load sample
            image, mask = self.load_sample(idx)
            #if self.transform:
            #    image, mask = self.transform(image, mask)
            image, mask = torch.from_numpy(image), torch.from_numpy(mask)
            if toggle:
                if self.images_cpu == None:
                    self.images_cpu = image.unsqueeze(0).cpu()
                    self.masks_cpu = mask.unsqueeze(0).cpu()
                else:
                    self.images_cpu = torch.cat((self.images_cpu, image.unsqueeze(0).cpu()), dim=0)
                    self.masks_cpu = torch.cat((self.masks_cpu, mask.unsqueeze(0).cpu()), dim=0)
                self.idx_to_device[idx] = ('cpu', self.images_cpu.shape[0]-1)
            else:
                if self.images_cuda == None:
                    self.images_cuda = image.unsqueeze(0).cuda()
                    self.masks_cuda = mask.unsqueeze(0).cuda()
                else:
                    self.images_cuda = torch.cat((self.images_cuda, image.unsqueeze(0).cuda()), dim=0)
                    self.masks_cuda = torch.cat((self.masks_cuda, mask.unsqueeze(0).cuda()), dim=0)
                self.idx_to_device[idx] = ('cuda', self.images_cuda.shape[0]-1)
            toggle = not toggle
            if idx % 50 == 1:
                print(f"cpu = {self.images_cuda.shape[0]} and cuda = {self.images_cpu.shape[0]}")
                self.print_memory()

    def transform(self, image, mask):
        # In the transform from albumentation we pass both the image and the mask together to make sure
        # they undergo the same transformation, e.g. this ensure both have the same random crop
        transformed = self.transform(image = image, mask = mask)
        image = transformed['image'].to(torch.float32)
        mask = transformed['mask']
        # Mask will be a one hot encoded matrix with dimensions 7 x 256 x 256
        # First dimenion corresponds to class label
        mask = np.apply_along_axis(lambda k: self.class_dict[tuple(k)], 2, mask)
        mask = (mask[..., None] == np.arange(7)).astype(mask.dtype)
        mask = np.transpose(mask, (2, 0, 1))
        mask = torch.tensor(mask).to(torch.float32)
        return image, mask

    def load_sample(self, idx):
            # Read image
            img_path = os.path.join(self.data_dir, self.meta_data.iloc[idx]['sat_image_path'])
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Read target mask
            mask_path = os.path.join(self.data_dir, self.meta_data.iloc[idx]['mask_path'])
            mask = cv2.imread(mask_path)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

            return image, mask
    def __getitem__(self, idx):
        image, mask = 0, 0
        if idx not in self.idx_to_device():
            image, mask = self.load_sample(idx)
            image, mask = torch.from_numpy(image).cuda(), torch.from_numpy(mask).cuda()
        else:
            device, index = self.idx_to_device[idx]
            if device == 'cuda':
                image, mask = self.images_cuda[index], self.masks_cuda[index]
            else:
                image, mask = self.images_cpu[index].cuda(), self.masks_cpu[index].cuda()
        image, mask = self.transform(image), self.transform(mask)
        return image, mask



In [7]:
# utils.py
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import pandas as pd

from PIL import Image
from torch.utils.data import SubsetRandomSampler
from torchmetrics import JaccardIndex
from torchmetrics.classification import Dice

def plot(sample, data_dir):

    plt.figure(figsize=(5,4))
    ax = plt.subplot(2,2,1)
    plt.imshow(np.asarray(Image.open(os.path.join(data_dir, sample['sat_image_path'].iloc[0]))))
    plt.gray()
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)

    ax = plt.subplot(2,2,2)
    plt.imshow(np.asarray(Image.open(os.path.join(data_dir, sample['mask_path'].iloc[0]))))
    plt.gray()
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)

    plt.show()

# TODO: make something like a dictionary for parametere to pass to data loader
def get_data_loaders(data_dir, transform, shuffle_dataset, test_split, random_seed, batch_size):

    device = torch.device("cuda")
    # Load metadata csv
    metadata = pd.read_csv(os.path.join(data_dir, 'metadata.csv'))

    # We need to filter for row where 'split' is 'train' because samples where
    # 'split' is 'valid' or 'test' have no target mask
    metadata = metadata[metadata['split'] == 'train']

    class_dict = pd.read_csv(os.path.join(data_dir, 'class_dict.csv'))

    classes = {}
    c = 0
    for i in class_dict.index:
        classes[tuple(class_dict.iloc[i,1:])] = c
        c += 1

    # We need to filter for row where 'split' is 'train' because samples where
    # 'split' is 'valid' or 'test' have no target mask

    dataset = SatelliteSet(meta_data=metadata, class_dict=classes, data_dir=data_dir, transform=transform)

    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(test_split * dataset_size))
    #TODO:Implement validation

    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, test_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                            sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)
    return train_loader, test_loader

def evaluate(model, writer, dataloader, epoch, device):
    iou_score = 0
    iou = JaccardIndex('multiclass', num_classes=7).to(device)
    dice_score = 0
    dice = Dice(num_classes=7).to(device)
    for x, y in dataloader:
        pred = model(x)
        pred = torch.argmax(pred, 1)
        y = torch.argmax(y, 1)
        # Calculate IOU
        iou_score += iou(pred, y)
        # Calculate DICE
        dice_score += dice(pred, y)

    writer.add_scalar('DICE Score', dice_score / len(dataloader), epoch)
    writer.add_scalar('IOU Score', iou_score / len(dataloader), epoch)

def get_iou_score(pred, y):
    iou_score = 0
    intersection = np.logical_and(y, pred)
    union = np.logical_or(y, pred)
    iou_score += np.sum(intersection) / np.sum(union)
    return iou_score

def get_dice_score(pred, y):
    dice_score = 0
    intersection = np.logical_and(y, pred)
    dice_score += np.sum(intersection) / (y.size() + pred.size())
    return dice_score

In [8]:
# Define constants

# directories
data_dir = 'deepglobe-land-cover-classification-dataset' # change to directory containing the data
trained_models = 'trained_models'
train_dir = 'train'
log_dir = 'runs'

In [9]:
# Training configuration (hyperparameters)

## Data
test_split = .2 #20% for test split
#valdation_split = .2 #20% for validation split
random_seed = np.random.seed()
shuffle_dataset = True

transform = alb.Compose([
    alb.RandomCrop(width=256, height=256),
    alb.HorizontalFlip(p=0.5),
    ToTensorV2()
    ],
    # we want the mask and the image to have the same augmentation (especially when we crop)
    # this way we pass the image and the mask simultaneously to the pipeline
    additional_targets={'image': 'image', 'mask': 'mask'}
    )

## model architecture
in_channels = 3
min_channels = 16
max_channels = 128
num_classes = 7

## Training
learning_rate = 0.1
batch_size = 20
epochs = 20

In [ ]:
from sys import breakpointhook
# Training

# setup training enviroment
device = torch.device("cuda")
#Labels


# init data loader/generator
train_dataloader, test_dataloader = get_data_loaders(data_dir, transform, shuffle_dataset, test_split, random_seed, batch_size)

# init model, optimizer and loss function
model = UNet(in_channels, min_channels, max_channels, num_classes)
model.to(device)
opt = torch.optim.SGD(model.parameters(), learning_rate)
loss_func = torch.nn.CrossEntropyLoss() #Dice(num_classes=num_classes)

# Set up summary writer for tensorboard
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(os.path.join(log_dir, current_time))

# start training
for epoch in range(1, epochs+1):
    print(f"Epoch: {epoch}")
    c = 1
    loss_avg = 0
    for x, y in train_dataloader:
        print(f"Batch number {c} of {len(train_dataloader)}")
        opt.zero_grad()
        pred = model(x)
        # pred has shape batch_size x num_classes x width x height
        # apply softmax to find what it thinks is the most likely class label
        pred = torch.softmax(pred, 1)
        # y has shape batch_size x num_classes x width x height
        # the num_classes is a one hot encoding where the index corresponds to a class
        # which can be looked up in the class_dict
        loss = loss_func(pred, y)
        loss_avg += loss
        loss.backward()
        opt.step()
        c += 1

    writer.add_scalar('Loss', loss_avg, epoch)

    # Evaluate model
    model.eval()
    evaluate(model, writer, test_dataloader, epoch, device)
    model.train()


In [ ]:
# Launch tensorboard
%load_ext tensorboard

%tensorboard --logdir runs/20231205-151422

In [ ]:
# List running tensorboard instances
tensorboard.notebook.list()

In [ ]:
# kill process with id
!kill -9 32287

In [ ]:
# Save model after training
torch.save(model.state_dict(), os.path.join(trained_models, current_time))

In [ ]:
# Model evaluation

model = UNet(0, 0, 0, 0)
model.load_state_dict(torch.load(PATH_To_Saved_Model))
model.eval()